<H1 style="text_align: center;">Credit Card Customer Analysis</H1>

 <br>**Extract meaningful insights that can inform strategic decisions to mitigate cardholder attrition.**
 
Business Goal: find the characteristics of a customer that is no longer an existing card holder



In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/credit-card-customers/BankChurners.csv


In [2]:
df = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')

In [3]:
df.shape

(10127, 23)

In [4]:
columns = ['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2','Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1']
df.drop(columns=columns, inplace = True)

In [5]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [6]:
df.Income_Category.unique()

array(['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
       '$120K +', 'Unknown'], dtype=object)

In [7]:
df[df['Card_Category']=='Gold'].Months_on_book.describe()

count    116.000000
mean      35.525862
std        6.618106
min       18.000000
25%       33.000000
50%       36.000000
75%       38.000000
max       55.000000
Name: Months_on_book, dtype: float64

In [8]:
df.Customer_Age.describe()

count    10127.000000
mean        46.325960
std          8.016814
min         26.000000
25%         41.000000
50%         46.000000
75%         52.000000
max         73.000000
Name: Customer_Age, dtype: float64

In [9]:
# Craeting a dataframe  with the gender and attrition flag to find ratio and percentage of churn by M/F
Attrition_by_Gender = df.groupby(['Gender','Attrition_Flag']).size()
Attrition_by_Gender = Attrition_by_Gender.reset_index(name="Count")

fig = px.bar(Attrition_by_Gender, x='Attrition_Flag', y='Count', color='Attrition_Flag', facet_col='Gender',
            category_orders={'Attrition_Flag': ['Existing Customer', 'Attrited Customer']})

fig.update_layout(title_text='Attrition by Gender',
                  xaxis_title='Attrition Flag',
                  yaxis_title='Count')

annotation_text = "Female Attrition Rate: 17.36%<br>"
annotation_text += "Male Attrition Rate: 14.6%<br><br>"
annotation_text += "Overall 2.76% difference"

fig.add_annotation(text=annotation_text,
                   xref="paper", yref="paper",
                   x=1.12, y=0.8,
                   showarrow=False,
                   font=dict(size=10))

fig.show()

**Quick Analysis:**
Women are more likely to attrit then men are in every category except the Silver Card Category. Most interesting 
is that in the Platinum category we see almost 50% of women leave the category versus men are more likely to stay with only a 9% attrition
percentage. In summary, the analysis indicates a slightly higher attrition rate among women compared to men, with a marginal difference of 3%.

<strong>Attrition Percentage By Card Category - Gender</strong>

| Blue        |     Gold    | Platinum      |  Silver |
| :---        |    :----:   |         :---: |    ---:
| F  17%     | F  21%     | F 44%  |  F 13% |
| M  14% | M    16%    | M 9%    |   M 15.7% |

<br>**Question is**: What are the benefits to being a part of each Card Category?<br>


* F    5358
* M    4769

In [10]:
df.dtypes

CLIENTNUM                     int64
Attrition_Flag               object
Customer_Age                  int64
Gender                       object
Dependent_count               int64
Education_Level              object
Marital_Status               object
Income_Category              object
Card_Category                object
Months_on_book                int64
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Credit_Limit                float64
Total_Revolving_Bal           int64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt               int64
Total_Trans_Ct                int64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
dtype: object

In [11]:
# Remove the Unkown Variable
Attrition_by_Income = df[df['Income_Category']!='Unknown']

Attrition_by_Income = Attrition_by_Income.groupby(['Income_Category','Attrition_Flag']).size()
Attrition_by_Income = Attrition_by_Income.reset_index(name="Count")
Attrition_by_Income

,Income_Category,Attrition_Flag,Count
0,$120K +,Attrited Customer,126
1,$120K +,Existing Customer,601
2,$40K - $60K,Attrited Customer,271
3,$40K - $60K,Existing Customer,1519
4,$60K - $80K,Attrited Customer,189
5,$60K - $80K,Existing Customer,1213
6,$80K - $120K,Attrited Customer,242
7,$80K - $120K,Existing Customer,1293
8,Less than $40K,Attrited Customer,612
9,Less than $40K,Existing Customer,2949


In [12]:
# Attrition Bar Graph By Income Category

custom_color_scale = ['#1f77b4', '#ff7f0e']


fig = px.bar(Attrition_by_Income, x='Attrition_Flag', y='Count',color='Attrition_Flag',
             facet_col='Income_Category',
            category_orders={'Attrition_Flag': ['Existing Customer', 'Attrited Customer']},
            color_discrete_sequence=custom_color_scale)



fig.update_layout(title_text='Attrition by Gender',
                  xaxis_title='Attrition',
                  yaxis_title='Count',
                  width=1100,  # Set the width of the plot
                  height=500 ) # Set the height of the plot)

facet_col_labels = ['$120K +', '$40K - 60K', '$60K - 80K', '$80K - 120K']

# Set facet subplot labels
for i, label in enumerate(facet_col_labels):
    fig['layout']['annotations'][i]['text'] = label

annotation_text = "120K+ Attrition Rate: 20.96%<br>"
annotation_text += "$40K - 60K Attrition Rate: 17.77%<br>"

fig.add_annotation(text=annotation_text,
                   xref="paper", yref="paper",
                   x=1.23, y=0.5,
                   showarrow=False,
                   font=dict(size=10))

fig.show()

#3561 -

# Percentage of Attrition by Income Category
| 120K +   |     40K - 60K  | 60K - 80K      |  80K - 120K | Less than $40K
| :---     |    :----:      |    :---:       |    :---:     |   ----:
|     17.33     |       15.14%     |    13.48%       |   15.77%      |  17.19%

<br>
The income bracket with the least rate of attition is 60K-80K and the bracket with the highest level of attrition is 120K +. Very close behind trails Less than $40K for the highest attrition.
<br>

In [13]:
df.Card_Category.unique()

array(['Blue', 'Gold', 'Silver', 'Platinum'], dtype=object)

In [14]:
# Group by customer Age - see if there's a correlation between age and attrition || age and any other column
gender_counts= df['Gender'].value_counts()
print(gender_counts)

# Get the ratio of males to females
male_count = gender_counts.get('M')
female_count = gender_counts.get('F')
ratio = round(male_count/female_count, 2)
print('\nRatio of men to female: ', ratio)

# Get the percentage of each 
total_count = df['Gender'].count()
male_perc = round((male_count/total_count)*100,2)
female_perc = round((female_count/total_count)*100,2)

print('\nTotal Count: ', total_count)
print('Male: ', male_perc)
print('Female: ', female_perc)

F    5358
M    4769
Name: Gender, dtype: int64

Ratio of men to female:  0.89

Total Count:  10127
Male:  47.09
Female:  52.91


Goal - find the attrition between the 'Blue', 'Gold', 'Silver', 'Platinum' category

In [15]:
# Delving into the Card Category
gold_category= df[df['Card_Category'] == 'Gold']
gold_category.describe()

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
count,1.160000e+02,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,7.381368e+08,45.439655,2.672414,35.525862,3.008621,2.310345,2.448276,28416.370690,1344.318966,27072.051724,0.772603,7685.612069,81.517241,0.706190,0.057103
std,3.756108e+07,6.577020,1.310664,6.618106,1.633857,1.074720,1.098179,8673.910045,774.882696,8648.962567,0.198927,4571.753917,26.713775,0.179573,0.059939
min,7.082268e+08,29.000000,0.000000,18.000000,1.000000,1.000000,0.000000,3735.000000,0.000000,2261.000000,0.196000,966.000000,22.000000,0.214000,0.000000
25%,7.130641e+08,41.000000,2.000000,33.000000,2.000000,1.000000,2.000000,22724.250000,813.000000,21144.250000,0.696000,4040.500000,66.750000,0.604750,0.029250
50%,7.168749e+08,46.000000,3.000000,36.000000,2.500000,2.000000,2.000000,34516.000000,1459.000000,32441.500000,0.742000,7652.500000,82.000000,0.723000,0.050500
75%,7.729482e+08,49.000000,4.000000,38.000000,4.000000,3.000000,3.000000,34516.000000,1952.250000,33238.500000,0.833250,9320.000000,100.250000,0.783500,0.070250
max,8.273330e+08,63.000000,5.000000,55.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,1.975000,16328.000000,131.000000,1.250000,0.395000


In [16]:
gender_counts_by_category = df.groupby(['Card_Category','Gender','Attrition_Flag']).size()
gender_counts_by_category = gender_counts_by_category.reset_index(name='Count')

custom_color_scale = ['#1f77b4', '#ff7f0e']

fig = px.bar(gender_counts_by_category, x='Gender', y='Count', color='Attrition_Flag', facet_col='Card_Category',
             category_orders={'Gender': ['F', 'M']}, color_discrete_sequence=custom_color_scale)

# Update layout for better visualization
fig.update_layout(title_text='Attrition by Card Category and Gender',
                  xaxis_title='Card Category',
                  yaxis_title='Count')

# Show the plot
fig.show()


In [17]:
gender_counts_by_category = df.groupby(['Gender','Card_Category','Attrition_Flag']).size()
gender_counts_by_category = gender_counts_by_category.reset_index(name='Count')
gender_counts_by_category

,Gender,Card_Category,Attrition_Flag,Count
0,F,Blue,Attrited Customer,890
1,F,Blue,Existing Customer,4211
2,F,Gold,Attrited Customer,8
3,F,Gold,Existing Customer,30
4,F,Platinum,Attrited Customer,4
5,F,Platinum,Existing Customer,5
6,F,Silver,Attrited Customer,28
7,F,Silver,Existing Customer,182
8,M,Blue,Attrited Customer,629
9,M,Blue,Existing Customer,3706


In [18]:
import matplotlib.pyplot as plt

Income_group= df[df['Income_Category'] == 'Unknown']
Income_group = Income_group.groupby(['Education_Level','Attrition_Flag', 'Gender']).size()


custom_color_scale = ['#1f77b4', '#ff7f0e']

fig = px.bar(gender_counts_by_category, x='Card_Category', y='Card_Category', color='Attrition_Flag', facet_col='Card_Category',
             category_orders={'Gender': ['F', 'M']}, color_discrete_sequence=custom_color_scale)

# Update layout for better visualization
fig.update_layout(title_text='Attrition by Card Category and Gender',
                  xaxis_title='Card Category',
                  yaxis_title='Count')

# Show the plot
fig.show()





# Display the plot
plt.show()

In [19]:
income_category = df.groupby(['Gender', 'Income_Category']).size()
print(income_category)

Gender  Income_Category
F       $40K - $60K        1014
        Less than $40K     3284
        Unknown            1060
M       $120K +             727
        $40K - $60K         776
        $60K - $80K        1402
        $80K - $120K       1535
        Less than $40K      277
        Unknown              52
dtype: int64


Group by income category and men/female

In [20]:
#Correlation Graph - Heat Map
import plotly.express as px
import plotly.graph_objects as go
import warnings

# Note: Deprecated Warnings
warnings.filterwarnings("ignore")

# Finding the correlation between variables with a heatmap

x=list(df.corr().columns)
y=list(df.corr().index)
values=np.array(df.corr().values)
fig = go.Figure(data=go.Heatmap(
    z=values,
    x=x,
    y=y,hoverongaps = False))

fig.update_layout(title='Correlation Heatmap')

fig.show()

In [21]:
# Correlation Graph - Scatter Plot
fig = go.Figure(data=go.Scatter(
    x=np.tile(x, len(y)),
    y=np.repeat(y, len(x)),
    mode='markers',
    marker=dict(
        color=values.flatten(),
        colorscale='Viridis',
        size=10,
        showscale=True
    )
))

fig.update_layout(
    title='Correlation Scatter Plot',
    xaxis=dict(tickvals=list(range(len(x))), ticktext=x, title='X Variables'),
    yaxis=dict(tickvals=list(range(len(y))), ticktext=y, title='Y Variables')
)

fig.show()

Correlation:
1. Customer Age and Months on Book
2. Total Revolving Balance andAverage Utilization Ration = 0.624

# Summary
Based on the varoius variables there's certain demographics that stick out. The goal for this analysis is customer churn for the credit card company; I split it into Income Categories, Age, Gender and Education Level. We discovered that the groups that are more likely to attrit are Females with a 2.75% higher then mens. As well as individuals in the income bracket of \\$120,000 and more, followed by the lowest income group which was individuals who make less than \\$40,000. 